# QATNE: Complete Implementation and Benchmarks\n\nThis notebook provides a complete, runnable implementation of the\nQuantum Adaptive Tensor Network Eigensolver (QATNE).\n\n## Features\n- Full mathematical derivations\n- Statistical analysis with confidence intervals\n- Comprehensive visualizations\n- Benchmarks against classical methods\n\n[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Tommaso-R-Marena/qatne/blob/main/notebooks/02_algorithm_implementation.ipynb)

## Setup\n\nFirst, install required packages (automatically handles Colab or local environment):

In [ ]:
# Install required packages\n!pip install -q qiskit qiskit-aer qiskit-ibm-runtime\n!pip install -q matplotlib seaborn scipy numpy\n!pip install -q openfermion pyscf openfermionpyscf\n\nimport numpy as np\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nfrom typing import Tuple, List\nimport time\nimport warnings\nwarnings.filterwarnings('ignore')\n\n# Set visualization style\nsns.set_style(\"whitegrid\")\nplt.rcParams['figure.figsize'] = (12, 8)\nplt.rcParams['font.size'] = 12\n\nprint(\"✓ Setup complete!\")

## Clone QATNE Repository (if running in Colab)

In [ ]:
# Clone repository and install package\n!git clone https://github.com/Tommaso-R-Marena/qatne.git 2>/dev/null || echo \"Repository already cloned\"\nimport sys\nsys.path.insert(0, './qatne')\n\nprint(\"✓ Repository ready!\")

## 1. Mathematical Framework\n\n### Theorem 1: Convergence Guarantee\n\n**Statement:** Let $\\mathcal{H}$ be a molecular Hamiltonian with ground state energy $E_0$. The QATNE algorithm converges to $\\epsilon$-accuracy of $E_0$ in $O(\\text{poly}(1/\\epsilon))$ iterations with probability at least $1 - \\delta$.\n\n**Key equations:**\n\nEnergy functional:\n$$E(\\boldsymbol{\\theta}) = \\langle\\psi(\\boldsymbol{\\theta})|\\mathcal{H}|\\psi(\\boldsymbol{\\theta})\\rangle$$\n\nGradient via parameter shift:\n$$\\frac{\\partial E}{\\partial \\theta_i} = \\frac{E(\\theta + \\pi/2 \\mathbf{e}_i) - E(\\theta - \\pi/2 \\mathbf{e}_i)}{2}$$\n\nError bound decomposition:\n$$\\Delta E_{\\text{total}} \\leq O(1/\\sqrt{N_{\\text{shots}}}) + O(\\epsilon_{\\text{gate}} \\cdot d) + O(1/\\chi^{\\alpha})$$

In [ ]:
# Demonstrate convergence theory\ndef theoretical_convergence(epsilon, delta=0.05):\n    \"\"\"Compute theoretical iteration bound\"\"\"\n    # From Theorem 1: T = O(1/epsilon^2 * log(1/delta))\n    constant = 10  # Algorithm-dependent constant\n    iterations = int(constant / (epsilon**2) * np.log(1/delta))\n    return iterations\n\n# Plot theoretical bounds\nepsilons = np.logspace(-6, -2, 20)\niterations = [theoretical_convergence(eps) for eps in epsilons]\n\nplt.figure(figsize=(10, 6))\nplt.loglog(epsilons, iterations, 'b-', linewidth=2, label='Theory: O(1/ε²)')\nplt.xlabel('Target accuracy ε')\nplt.ylabel('Required iterations')\nplt.title('QATNE Convergence: Theoretical Bounds')\nplt.legend()\nplt.grid(True, alpha=0.3)\nplt.show()\n\nprint(f\"For chemical accuracy (ε=1.6e-3 Ha): {theoretical_convergence(1.6e-3)} iterations\")

## 2. Create Molecular Hamiltonian\n\nWe use the H₂ molecule as our benchmark system.

In [ ]:
from qatne.benchmarks.molecular_systems import create_h2_hamiltonian\n\n# Create H2 Hamiltonian\nbond_length = 0.735  # Angstroms (equilibrium)\nH_matrix, num_qubits, exact_energy = create_h2_hamiltonian(bond_length)\n\nprint(f\"=\" * 60)\nprint(f\"H₂ MOLECULE HAMILTONIAN\")\nprint(f\"=\" * 60)\nprint(f\"Bond length:         {bond_length} Å\")\nprint(f\"Number of qubits:    {num_qubits}\")\nprint(f\"Hilbert space dim:   {2**num_qubits}\")\nprint(f\"FCI ground energy:   {exact_energy:.10f} Ha\")\nprint(f\"Hamiltonian norm:    {np.linalg.norm(H_matrix, ord=2):.6f}\")

## 3. Initialize QATNE Solver

In [ ]:
from qatne.algorithms.qatne_solver import QATNESolver\n\n# Initialize solver\nsolver = QATNESolver(\n    hamiltonian=H_matrix,\n    num_qubits=num_qubits,\n    max_bond_dim=16,\n    convergence_threshold=1e-6,\n    shots=8192\n)\n\nprint(\"✓ QATNE solver initialized\")\nprint(f\"  Max bond dimension: {solver.max_bond_dim}\")\nprint(f\"  Convergence threshold: {solver.convergence_threshold}\")\nprint(f\"  Shots per measurement: {solver.shots}\")

## 4. Run Optimization\n\nThis may take 2-5 minutes depending on your hardware.

In [ ]:
# Set random seed for reproducibility\nnp.random.seed(42)\n\n# Solve for ground state\nprint(\"Starting QATNE optimization...\\n\")\nstart_time = time.time()\nground_energy, optimal_params = solver.solve(\n    max_iterations=200,\n    optimizer='adam'\n)\nend_time = time.time()\n\nprint(f\"\\n{\"=\" * 60}\")\nprint(f\"OPTIMIZATION COMPLETE\")\nprint(f\"{\"=\" * 60}\")\nprint(f\"Ground energy (QATNE): {ground_energy:.10f} Ha\")\nprint(f\"Exact energy (FCI):    {exact_energy:.10f} Ha\")\nprint(f\"Absolute error:        {abs(ground_energy - exact_energy):.2e} Ha\")\nprint(f\"Relative error:        {abs(ground_energy - exact_energy) / abs(exact_energy) * 100:.4f}%\")\nprint(f\"Chemical accuracy:     {'YES' if abs(ground_energy - exact_energy) < 1.6e-3 else 'NO'}\")\nprint(f\"Total time:            {end_time - start_time:.2f} seconds\")\nprint(f\"Iterations:            {len(solver.energy_history)}\")\nprint(f\"Final bond dimension:  {solver.tensor_network.bond_dim}\")